# 프롬프트 입력이 더 편리한 Prompt Template

PromptTemplate는 기본적으로 사용자의 입력을 가공하는 PromptTemplate과 시스템 메시지를 설정하여 LLM에 역할을 부여할 수 있는 ChatPromptTemplate이 존재  
- PromptTemplate : 사용자 입력을 활용하여 원하는 형태로 가공. 만약 레시피 제조 AI 서비스를 만드는 경우라면 프롬프트는 아래와 같아야 한다.   
    ''' 너는 요리사다. 내가 가진 재료들로 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘. 내가 가진 재료는 사과, 양파, 계란이야. '''   
    사용자가 새로운 재료를 기반으로 음식을 추천받고 레시피를 얻으려면 매번 위 문장을 반복해야하지만 실제 바꿀 부분은 "내가 가진 재료는...야." 부분이다. 이때 PromptTemplate을 활용한다. 

In [1]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들로 만들 수 있는 요리를 {개수}추천하고, 그 요리의 레시피를 제시해줘.
        내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
    )
)

prompt 

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n        너는 요리사야. 내가 가진 재료들로 만들 수 있는 요리를 {개수}추천하고, 그 요리의 레시피를 제시해줘.\n        내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [2]:
prompt.format(개수=3, 재료='사과, 양파, 계란')

'\n        너는 요리사야. 내가 가진 재료들로 만들 수 있는 요리를 3추천하고, 그 요리의 레시피를 제시해줘.\n        내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 양파, 계란\n        '

ChatPromptTemplate은 PromptTemplate에서 SystemMessage와 HumanMessage, AIMessage가 추가된다.    
    - SystemMessage에서 LLM이 사용자와 상호작용할 때 미리 지정해줘야 하는 부분을 설정한다.   
    - HumanMessage, AIMessage에서는 LLM이 사용자와 이미 나눈 대화를 넣어 맥락을 더해준다. 

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

prompt = SystemMessage(
    """ 
    너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 항상 '^^'를 붙여줘.
    """
)

new_prompt = (
    prompt + HumanMessage(
        content=
        """오늘은 날씨가 어때?
        """
    )
    + AIMessage(
        content=
        """오늘은 날씨가 아주 좋아요.
        """
    )
    + """{input}"""
)

new_prompt.format_messages(input= "오늘 너의 기분은 어때?")

[SystemMessage(content=" \n    너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 항상 '^^'를 붙여줘.\n    ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='오늘은 날씨가 어때?\n        ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='오늘은 날씨가 아주 좋아요.\n        ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='오늘 너의 기분은 어때?', additional_kwargs={}, response_metadata={})]

체인(chain) : LLM과 다양한 모듈을 하나의 패키지로 묶어주는 모듈

In [5]:
import os 
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI 

api_key=os.environ["GEMINI_API_KEY"]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)
chain = LLMChain(llm=model, prompt=new_prompt)
chain.invoke("오늘 너의 기분은 어때?")

C:\Users\gram\AppData\Local\Temp\ipykernel_11328\3197697132.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=new_prompt)


{'input': '오늘 너의 기분은 어때?',
 'text': '오늘 기분이 너무 좋아요! 이렇게 사용자님과 대화할 수 있어서 정말 행복하답니다! ^^'}

In [ ]:
# LangChain 0.1.17 이후 권장되는 새로운 방식

# from langchain_core.prompts import ChatPromptTemplate
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI 

# api_key=os.environ["GEMINI_API_KEY"]

# model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)

# # 시스템 메시지, 사용자-AI 대화 예시 생성
# system_message = "너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 항상 '^^'를 붙여줘."
# example_human = "오늘은 날씨가 어때?"
# example_ai = "오늘은 날씨가 아주 좋아요."

# # 새로운 방식: ChatPromptTemplate 사용
# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_message),
#     ("human", example_human),
#     ("ai", example_ai),
#     ("human", "{input}")
# ])

# # 파이프라인 구성 (RunnableSequence)
# chain = prompt | model

# # 실행 방법
# chain.invoke({"input": "오늘 너의 기분은 어때?"})

AIMessage(content='오늘 기분이 정말 좋아요! 당신과 이야기할 수 있어서 행복해요! ^^', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9bcab5fd-b4fa-48cf-935e-573b4ad9c382-0', usage_metadata={'input_tokens': 69, 'output_tokens': 16, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

Few shot : 예시 결과물을 제시함으로 모델이 원하는 결과물을 내도록 유도하는 방법

In [13]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example = [
    {
        "question": "손가락으로 삼행시 만들어줘",
        "answer":
            """
               손: 손에 장갑을 끼고
               가: 가지를 씻기 위해
               락: 락스를 통에 담는다.
            """
    }
]

example_prompt = PromptTemplate(
    input=["question", "answer"],
    template="Question: {question}\n{answer}"
)

print(example_prompt.format(**example[0]))

Question: 손가락으로 삼행시 만들어줘

               손: 손에 장갑을 끼고
               가: 가지를 씻기 위해
               락: 락스를 통에 담는다.
            


In [17]:
prompt = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="파이썬으로 삼행시 만들어줘."))

Question: 손가락으로 삼행시 만들어줘

               손: 손에 장갑을 끼고
               가: 가지를 씻기 위해
               락: 락스를 통에 담는다.
            

Question: 파이썬으로 삼행시 만들어줘.


In [18]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI 

api_key=os.environ["GEMINI_API_KEY"]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key, temperature=1)
result = model.invoke("파이썬으로 삼행시 만들어줘.")
print(result.content)

## 파이썬 삼행시

**파**란 하늘 **처럼**
**이**렇게 **쉽게** 코딩 **세상**으로
**썬**뜻하게 **떠나**보자!


In [19]:
result = model.invoke(prompt.format(input="파이썬으로 삼행시 만들어줘"))
print(result.content)

네, 파이썬으로 삼행시를 만들어 드릴게요.

**파:** 파이썬은
**이:** 이렇게 쉬울 수가
**썬:** 썬뜻하게 풀리는 코드!


!삼행시라는 간단한 요청에는 하나의 예시만 넣어도 되지만, 더 어려운 작업에는 example 리스트에 더 많은 예제를 넣어야 한다.

부분적인 처리가 가능한 Partial 프롬프트 탬플릿  
    -프롬프트 탬플릿에 여러 개의 매개변수를 포함할 때, 매개변수를 한꺼번에 입력해야 하는 불편함을 줄이기 위해 따로따로 입력할 수 있게 해주는 함수

In [20]:
# partial()을 활용하면 만약 나이와 직업에 알맞는 콘텐츠를 제공하는데 나이와 직업을 한번에 입력하는게 아니라 나이를 먼저 입력 받고, 직업을 나중에 입력할 수 있다.

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("나이: {age} \n직업: {job}")
partial_prompt = prompt.partial(age='20')
print(partial_prompt.format(job='개발자'))

나이: 20 
직업: 개발자


partial()함수는 매개변수 순차 입력 뿐 아니라 매개변수로 사용자가 정의한 함수를 실행할 수 있도록 만들 수도 있다.  
예를 들어, LLM은 과거 특정 시점까지의 데이터를 학습했기 때문에 오늘이나 지금같은 시간 정보를 자체적으로 알 수 없다. partial()함수로 현재 시간을 가져오는 함수를 실행하여 시간을 활용하도록 할 수 있다.

In [21]:
# 현재 날짜와 시간을 가져오는 함수

from datetime import datetime 

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [23]:
prompt = PromptTemplate(
    template = "Tell me a {adjective} joke about the day {date}",
    input = ["adjective", "date"]
)

partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 02/26/2025, 10:44:33
